In [1]:
import torch
from tqdm import tqdm
from safetensors import safe_open
from safetensors.torch import save_file
import sys
sys.path.append("../AnyDoor/")
sys.path.append("../")


### DDIM Sampler


In [2]:
x = torch.load("../tests/tensors/x.pt",weights_only=True)
initial_latents = torch.randn(1,4,32,32)
object_embedding = torch.load("../tests/tensors/object_embedding.pt",weights_only=True)
negative_object_embedding = torch.load("../tests/tensors/negative_object_embedding.pt",weights_only=True)
control = torch.load("../tests/tensors/control_features.pt",weights_only=True)
inference_steps = 10
scale = 5.0

KeyboardInterrupt: 

In [ ]:
weights_anydoor = torch.load("../ckpt/unet.ckpt",weights_only=True)
# weights_refiners = torch.load("../ckpt/unet_refiners.ckpt",weights_only=True)

In [ ]:
from anydoor_original.cldm import model as cldm

No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
Loaded model config from [../src/anydoor_original/configs/anydoor.yaml]
Model loaded successfully


In [ ]:
cldm.model.diffusion_model.load_state_dict(weights_anydoor)

<All keys matched successfully>

In [ ]:
from cldm.ddim_hacked import DDIMSampler
sampler = DDIMSampler(cldm)

In [ ]:
with torch.no_grad():
    mocked_control_image = torch.zeros(1, 4, 32, 32)
    cond = {
        "c_concat": control, ## Not used
        "c_crossattn": [object_embedding],
    }
    un_cond = {
        "c_concat": control, ## Not used
        "c_crossattn": [negative_object_embedding],
    }

    samples, intermediates = sampler.sample(
        S=inference_steps,
        batch_size=1,
        shape=(4, 32, 32),
        conditioning=cond,
        x_T=initial_latents.clone(),
        verbose=False,
        unconditional_guidance_scale=scale,
        unconditional_conditioning=un_cond,
    )

Data shape for DDIM sampling is (1, 4, 32, 32), eta 0.0
Running DDIM Sampling with 10 timesteps


DDIM Sampler:  10%|█         | 1/10 [00:02<00:18,  2.08s/it]

tensor(67.6540)


DDIM Sampler:  20%|██        | 2/10 [00:04<00:15,  1.99s/it]

tensor(71.5997)


...

tensor(87.3436)


DDIM Sampler:  90%|█████████ | 9/10 [00:17<00:01,  1.90s/it]

tensor(86.2254)


DDIM Sampler: 100%|██████████| 10/10 [00:19<00:00,  1.95s/it]

tensor(86.1132)


In [ ]:
from anydoor_refiners.model import AnyDoor,solver_params
from refiners.foundationals.latent_diffusion.solvers import DDIM
from tests.mocks import DINOv2EncoderMock,ControlNetMock,AnydoorAutoencoderMock

refiners_model = AnyDoor(
    lda=AnydoorAutoencoderMock(),
    object_encoder=DINOv2EncoderMock(object_embedding,negative_object_embedding),
    control_model=ControlNetMock(control),
    solver=DDIM(inference_steps,params=solver_params)
)
# refiners_model.unet.load_state_dict(weights_refiners)


In [ ]:
import json
from utils.weight_mapper import get_converted_state_dict

with open("../tests/weights_mapping/unet.json", "r") as f:
    weight_mapping = json.load(f)
converted_state_dict = get_converted_state_dict(
    source_state_dict=cldm.model.diffusion_model.state_dict(),
    target_state_dict=refiners_model.unet.state_dict(),
    mapping=weight_mapping,
)
refiners_model.unet.load_state_dict(converted_state_dict)

<All keys matched successfully>

In [ ]:
with torch.no_grad():

    y = initial_latents.clone()
    conditionning = refiners_model.compute_conditionning(object=torch.zeros(1),background=torch.zeros(1))
    for s in tqdm(refiners_model.steps):
        y = refiners_model.forward(y,step=s,conditionning=conditionning,condition_scale=scale)
        print(torch.norm(y))

 10%|█         | 1/10 [00:01<00:09,  1.06s/it]

tensor(67.1199)


 20%|██        | 2/10 [00:02<00:08,  1.06s/it]

tensor(70.5397)


 30%|███       | 3/10 [00:03<00:07,  1.09s/it]

...

tensor(66.9328)


 90%|█████████ | 9/10 [00:13<00:01,  1.85s/it]

tensor(61.8021)


100%|██████████| 10/10 [00:15<00:00,  1.54s/it]

tensor(61.4083)


In [ ]:
torch.norm(y-samples[-1]),torch.norm(y),torch.norm(samples[-1])

(tensor(58.3949), tensor(61.4083), tensor(86.1132))

# LDA

In [2]:
import torch
import sys
from omegaconf import OmegaConf

sys.path.append("./AnyDoor/")
from ldm.util import instantiate_from_config

conf = OmegaConf.load("src/anydoor_original/configs/anydoor.yaml")

lda_anydoor = instantiate_from_config(conf.model.params.first_stage_config)


No module 'xformers'. Proceeding without it.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [3]:
lda_anydoor.load_state_dict(torch.load("ckpt/lda_anydoor.ckpt"))

/var/folders/_s/grxhyczx135dcrhc7pzfhkxr0000gn/T/ipykernel_4580/2496509189.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lda_anydoor.load_state_dict(torch.load("ckpt/l

<All keys matched successfully>

In [4]:
from anydoor_refiners.model import AnydoorAutoencoder

lda_refiners = AnydoorAutoencoder()

In [6]:
lda_refiners = lda_refiners.load_from_safetensors("ckpt/anydoor_refiners_safetensors/lda.safetensors")

In [7]:
# Print nb of trainable parameters of the two models

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(lda_anydoor))
print(count_parameters(lda_refiners))


83653863
83653863


In [15]:
with torch.no_grad():
    img = torch.randn(1,3,256,256)
    y1 = lda_refiners.forward(img)
    y2 = lda_anydoor.forward(img)

In [19]:
torch.norm(y1 - y2[0])

tensor(207.3743)

In [16]:
y2[0].shape

torch.Size([1, 3, 256, 256])

In [17]:
from refiners.conversion.model_converter import ModelConverter

converter = ModelConverter(source_model=lda_anydoor,target_model=lda_refiners)

In [18]:
with torch.no_grad():
    img = torch.randn(1,3,256,256)
    converter.run(source_args=(img,),target_args=(img,))

Models do not have the same number of basic layers:
  <class 'torch.nn.modules.conv.Conv2d'>: Source 72 - Target 64
  <class 'torch.nn.modules.linear.Linear'>: Source 0 - Target 8
Conversion failed at stage 1


In [7]:
y.shape

torch.Size([1, 3, 256, 256])